# Mini Proyecto 3 


## Instalación de ambiente
Debe subir el archivo "miniproyecto_installer.py"

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving miniproyecto3_installer.py to miniproyecto3_installer.py
User uploaded file "miniproyecto3_installer.py" with length 2561422 bytes


In [2]:
exec(open('miniproyecto3_installer.py').read())

Active services:
2048 NodeManager
1907 DataNode
1667 NameNode
2104 JobHistoryServer
1577 ResourceManager
2158 Jps



## Actividad 1
Visualización de datos en MySQL: *Hate Speech and Offensive Language Dataset*


In [ ]:
!mysql -u root -p

Enter password: 
Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 31
Server version: 5.7.32-0ubuntu0.18.04.1 (Ubuntu)

Copyright (c) 2000, 2020, Oracle and/or its affiliates. All rights reserved.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
| testdb             |
+--------------------+
5 rows in set (0.00 sec)

mysql> use testdb
Reading table information for completion of table and column names
You can turn off this feature to get a quicker startup with -A

Database changed
mysql> select * from hate_speech limit 5;
+----------+-------+-------------+--------------------+---------+-------+------------

## Actividad 2
Inserción de datos con Sqoop. El password de MySQL es "password" (sin comillas). 

In [3]:
!sqoop import-all-tables --connect jdbc:mysql://localhost/testdb --username root -P --hive-import

20/11/09 00:52:26 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
Enter password: 
20/11/09 00:52:31 INFO tool.BaseSqoopTool: Using Hive-specific delimiters for output. You can override
20/11/09 00:52:31 INFO tool.BaseSqoopTool: delimiters with --fields-terminated-by, etc.
20/11/09 00:52:31 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
20/11/09 00:52:32 INFO tool.CodeGenTool: Beginning code generation
20/11/09 00:52:32 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `hate_speech` AS t LIMIT 1
20/11/09 00:52:32 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `hate_speech` AS t LIMIT 1
20/11/09 00:52:32 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /content/hadoop
Note: /tmp/sqoop-root/compile/78e06

## Actividad 3
Visualización de datos con Hive

In [5]:
%%writefile hive_mostrar.sql
SELECT *
FROM hate_speech
limit 10;

Overwriting hive_mostrar.sql


In [6]:
!hive -f hive_mostrar.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.7.0/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
OK
hate_speech.tweet_id	hate_speech.count	hate_speech.hate_speech	hate_speech.offensive_language	hate_speech.neither	hate_speech.class	hate_speech.tweet
0	3	0	0	3	2	!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1	3	0	3	0	1	!!!!! RT @mleew17: boy dats cold...tyga

## Actividad 4
Lectura de datos con Spark SQL y separación train/test

In [7]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.enableHiveSupport().appName("MP-3").getOrCreate()

In [8]:
df_hate = spark.sql("select * from hate_speech where length(tweet) > 1 AND tweet is not NULL")
df_hate = df_hate.withColumn('label', df_hate['class']+1)
df_hate.show(10)
print(df_hate.count())
training, testing = df_hate.randomSplit([0.95, 0.05])
print(training.count(), testing.count())

+--------+-----+-----------+------------------+-------+-----+--------------------+-----+
|tweet_id|count|hate_speech|offensive_language|neither|class|               tweet|label|
+--------+-----+-----------+------------------+-------+-----+--------------------+-----+
|       0|    3|          0|                 0|      3|    2|!!! RT @mayasolov...|    3|
|       1|    3|          0|                 3|      0|    1|!!!!! RT @mleew17...|    2|
|       2|    3|          0|                 3|      0|    1|!!!!!!! RT @UrKin...|    2|
|       3|    3|          0|                 2|      1|    1|!!!!!!!!! RT @C_G...|    2|
|       4|    6|          0|                 6|      0|    1|!!!!!!!!!!!!! RT ...|    2|
|       5|    3|          1|                 2|      0|    1|"!!!!!!!!!!!!!!!!...|    2|
|       6|    3|          0|                 3|      0|    1|"!!!!!!""@__Brigh...|    2|
|       7|    3|          0|                 3|      0|    1|!!!!&#8220;@selfi...|    2|
|       8|    3|     

## Actividad 5
Preprocesamiento de dataset de entrenamiento, con funciones de _tokenización_ y remoción de _stopwords_ 

In [10]:
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover

tokenizer = RegexTokenizer(pattern="\\W+", inputCol='tweet', outputCol='words' )

stopWords=StopWordsRemover.loadDefaultStopWords('english')
stopWordsRemover = StopWordsRemover(stopWords = stopWords, inputCol='words', outputCol='clean_words' )

In [11]:
# tokenizer
training_words = tokenizer.transform(training)

In [12]:
# mostrar las columnas "words" y "tweet" de training_words
training_words.select(['tweet','words']).show(5)

+--------------------+--------------------+
|               tweet|               words|
+--------------------+--------------------+
|!!! RT @mayasolov...|[rt, mayasolovely...|
|!!!!! RT @mleew17...|[rt, mleew17, boy...|
|!!!!!!! RT @UrKin...|[rt, urkindofbran...|
|!!!!!!!!! RT @C_G...|[rt, c_g_anderson...|
|!!!!!!!!!!!!! RT ...|[rt, shenikarober...|
+--------------------+--------------------+
only showing top 5 rows



In [13]:
# stopWordsRemover
training_clean = stopWordsRemover.transform(training_words)

In [14]:
from pyspark.sql.functions import size

# Se usan finalmente aquellas frases con más de 3 palabras que no sean stopwords
train_words = training_clean.filter(size(training_clean['clean_words']) > 3)
train_words.show(n=5, truncate=False)

+--------+-----+-----------+------------------+-------+-----+--------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------+
|tweet_id|count|hate_speech|offensive_language|neither|class|tweet                                                                                                                                       |label|words                                                                                                                                                     |clean_words                                                                                 |
+--------+-----+-----------+------------------+-------+-----+-------------------------

## Actividad 6
Entrenamiento de modelo Word2Vec y transformación a features numéricos

In [15]:
from pyspark.ml.feature import Word2Vec

model_w2v = Word2Vec(vectorSize=32, minCount=0, inputCol="clean_words", outputCol="features").fit(train_words)
train_features = model_w2v.transform(train_words)

In [16]:
# mostrar Dataframe final que se usará para entrenamiento
train_features.show()

+--------+-----+-----------+------------------+-------+-----+--------------------+-----+--------------------+--------------------+--------------------+
|tweet_id|count|hate_speech|offensive_language|neither|class|               tweet|label|               words|         clean_words|            features|
+--------+-----+-----------+------------------+-------+-----+--------------------+-----+--------------------+--------------------+--------------------+
|       0|    3|          0|                 0|      3|    2|!!! RT @mayasolov...|    3|[rt, mayasolovely...|[rt, mayasolovely...|[-0.0403613432620...|
|       1|    3|          0|                 3|      0|    1|!!!!! RT @mleew17...|    2|[rt, mleew17, boy...|[rt, mleew17, boy...|[-0.0091969449870...|
|       2|    3|          0|                 3|      0|    1|!!!!!!! RT @UrKin...|    2|[rt, urkindofbran...|[rt, urkindofbran...|[-0.0063409162491...|
|       3|    3|          0|                 2|      1|    1|!!!!!!!!! RT @C_G...|    2|

## Actividad 7
Entrenamiento de modelo clasificador RandomForestClassifier

In [17]:
from pyspark.ml.classification import RandomForestClassifier

algorithm = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=200)
model = algorithm.fit(train_features)


In [18]:
# obtener Dataframes de testing

testing_words = tokenizer.transform(testing)
testing_clean = stopWordsRemover.transform(testing_words)
test_words = testing_clean.filter(size(testing_clean['clean_words']) > 3)
test_features = model_w2v.transform(test_words)

In [19]:
# obtener predicciones

predictions = model.transform(train_features)

In [20]:
# mostrar label, predicciones, y el tweet original

predictions.select(['class','prediction','tweet']).show(10)

+-----+----------+--------------------+
|class|prediction|               tweet|
+-----+----------+--------------------+
|    2|       2.0|!!! RT @mayasolov...|
|    1|       2.0|!!!!! RT @mleew17...|
|    1|       2.0|!!!!!!! RT @UrKin...|
|    1|       2.0|!!!!!!!!! RT @C_G...|
|    1|       2.0|!!!!!!!!!!!!! RT ...|
|    1|       2.0|"!!!!!!!!!!!!!!!!...|
|    1|       2.0|"!!!!!!""@__Brigh...|
|    1|       2.0|!!!!&#8220;@selfi...|
|    1|       2.0|""" &amp; you mig...|
|    1|       3.0|""" @rhythmixx_ :...|
+-----+----------+--------------------+
only showing top 10 rows



In [21]:
# evaluación con métrica de accuracy

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("Test accuracy: ", "%2.1f%%" % (accuracy*100,))

Test accuracy:  84.3%
